In [46]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [49]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [50]:
STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  

STARTINGCSV=pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,shorthandedPoints,shorthandedGoals,opponentAbbrev,pim,toi,playerId,firstName,lastName
0,2023020564,WPG,R,2023-12-31,0,1,{'default': 'Jets'},{'default': 'Wild'},1,0,0,0,0,0,1,25,0,0,MIN,0,19:32,8475455,Brenden,Dillon
1,2023020555,WPG,H,2023-12-30,0,0,{'default': 'Jets'},{'default': 'Wild'},0,0,0,0,0,0,1,21,0,0,MIN,7,14:17,8475455,Brenden,Dillon
2,2023020538,WPG,R,2023-12-27,0,0,{'default': 'Jets'},{'default': 'Blackhawks'},0,0,0,0,0,0,0,23,0,0,CHI,0,17:30,8475455,Brenden,Dillon
3,2023020512,WPG,H,2023-12-22,0,0,{'default': 'Jets'},{'default': 'Bruins'},0,-1,0,0,0,0,0,29,0,0,BOS,0,18:21,8475455,Brenden,Dillon
4,2023020495,WPG,H,2023-12-20,0,0,{'default': 'Jets'},{'default': 'Red Wings'},0,-1,0,0,0,0,1,24,0,0,DET,0,20:10,8475455,Brenden,Dillon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,2023020601,SJS,H,2024-01-04,1,0,{'default': 'Sharks'},{'default': 'Jets'},1,0,0,0,0,0,3,19,0,0,WPG,0,15:09,8482222,Alexander,Barabanov
1672,2023020586,SJS,H,2024-01-02,1,0,{'default': 'Sharks'},{'default': 'Red Wings'},1,1,0,0,0,0,1,18,0,0,DET,0,11:50,8482222,Alexander,Barabanov
1673,2023020613,SJS,H,2024-01-06,0,0,{'default': 'Sharks'},{'default': 'Maple Leafs'},0,-2,0,0,0,0,0,20,0,0,TOR,0,17:58,8482667,William,Eklund
1674,2023020601,SJS,H,2024-01-04,0,1,{'default': 'Sharks'},{'default': 'Jets'},1,1,0,0,0,0,0,22,0,0,WPG,0,19:55,8482667,William,Eklund


In [58]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,Winnipeg,Jets,WPG
1,NY Rangers,Rangers,NYR
2,Boston,Bruins,BOS
3,Vancouver,Canucks,VAN
4,Colorado,Avalanche,COL
5,Florida,Panthers,FLA
6,Vegas,Golden Knights,VGK
7,Dallas,Stars,DAL
8,Carolina,Hurricanes,CAR
9,Toronto,Maple Leafs,TOR


In [51]:
playerStats=STARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','shots','opponentAbbrev']]

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('teamAbbrev').GameNumber.transform(np.max)

playerStats['SeasonAvgSOG']= playerStats.groupby(['playerId']).shots.transform(np.sum)/playerStats['GP']
playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['shots']))

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)

playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).shots.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,39,39,0.923077,0.464458,1,0.6
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,38,39,0.923077,0.464458,1,0.6
2,WPG,8475455,Brenden,Dillon,2023-12-27,0,CHI,37,39,0.923077,1.105324,1,0.6
3,WPG,8475455,Brenden,Dillon,2023-12-22,0,BOS,36,39,0.923077,1.105324,1,0.6
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,35,39,0.923077,0.464458,1,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,SJS,8482222,Alexander,Barabanov,2024-01-04,3,WPG,2,40,0.700000,0.817273,0,0.0
1672,SJS,8482222,Alexander,Barabanov,2024-01-02,1,DET,1,40,0.700000,0.464458,0,0.0
1673,SJS,8482667,William,Eklund,2024-01-06,0,TOR,3,40,1.500000,1.105324,0,2.6
1674,SJS,8482667,William,Eklund,2024-01-04,0,WPG,2,40,1.500000,1.105324,0,2.6


In [52]:
res = playerStats.groupby(['playerId'])["shots"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['shots'])&(res['shots']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['shots'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).shots.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,39,39,0.923077,0.464458,1,0.6,21,21,1.285714
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,38,39,0.923077,0.464458,1,0.6,20,21,1.285714
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,35,39,0.923077,0.464458,1,0.6,19,21,1.285714
6,WPG,8475455,Brenden,Dillon,2023-12-16,1,COL,33,39,0.923077,0.464458,0,0.6,18,21,1.285714
7,WPG,8475455,Brenden,Dillon,2023-12-13,2,LAK,32,39,0.923077,0.176407,0,0.6,17,21,1.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21672,SJS,8481567,Henry,Thrun,2024-01-06,1,TOR,3,40,0.525000,0.464458,0,0.0,2,11,1.636364
21674,SJS,8481567,Henry,Thrun,2024-01-02,2,DET,1,40,0.525000,0.176407,0,0.0,1,11,1.636364
21676,SJS,8482222,Alexander,Barabanov,2024-01-04,3,WPG,2,40,0.700000,0.817273,0,0.0,2,12,1.666667
21677,SJS,8482222,Alexander,Barabanov,2024-01-02,1,DET,1,40,0.700000,0.464458,0,0.0,1,12,1.666667


In [53]:
shotSummary=ShotsNoOutliers[['teamAbbrev',	'playerId',	'firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots', 'NoOutliersAvgSOG']].drop_duplicates()
shotSummary.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,NoOutliersAvgSOG
0,BOS,8477956,David,Pastrnak,4.973684,5.8,5.233333
1,TBL,8476453,Nikita,Kucherov,4.341463,3.8,4.838710
2,TOR,8477939,William,Nylander,4.405405,4.0,4.714286
3,COL,8477492,Nathan,MacKinnon,4.475000,5.0,4.645161
4,NJD,8481559,Jack,Hughes,3.631579,0.0,4.600000
5,TOR,8479318,Auston,Matthews,4.351351,3.6,4.322581
6,OTT,8480801,Brady,Tkachuk,4.542857,5.4,4.310345
7,NYR,8478550,Artemi,Panarin,4.184211,5.4,4.281250
8,NSH,8476887,Filip,Forsberg,3.850000,5.8,4.096774
9,VGK,8478403,Jack,Eichel,4.125000,4.4,3.939394


In [55]:
import requests
todaysDate=str(pd.to_datetime('today').normalize().date())

link = BaseURL+'/v1/club-schedule/DET/week/'+todaysDate
f = requests.get(link)
df=pd.DataFrame(f.json()['games'])
df


,id,season,gameType,gameDate,venue,neutralSite,startTimeUTC,easternUTCOffset,venueUTCOffset,venueTimezone,gameState,gameScheduleState,tvBroadcasts,awayTeam,homeTeam,periodDescriptor,gameOutcome,winningGoalie,winningGoalScorer,threeMinRecap,gameCenterLink,ticketsLink
0,2023020620,20232024,2,2024-01-07,{'default': 'Honda Center'},False,2024-01-08T01:00:00Z,-05:00,-08:00,America/Los_Angeles,FINAL,OK,"[{'id': 282, 'market': 'N', 'countryCode': 'CA...","{'id': 17, 'placeName': {'default': 'Detroit'}...","{'id': 24, 'placeName': {'default': 'Anaheim'}...","{'number': 3, 'periodType': 'REG'}",{'lastPeriodType': 'REG'},"{'playerId': 8479312, 'firstInitial': {'defaul...","{'playerId': 8479992, 'firstInitial': {'defaul...",/video/recap-red-wings-at-ducks-1-7-24-6344459...,/gamecenter/det-vs-ana/2024/01/07/2023020620,NaN
1,2023020640,20232024,2,2024-01-11,{'default': 'Little Caesars Arena'},False,2024-01-12T00:00:00Z,-05:00,-05:00,America/Detroit,FUT,OK,"[{'id': 289, 'market': 'A', 'countryCode': 'CA...","{'id': 22, 'placeName': {'default': 'Edmonton'...","{'id': 17, 'placeName': {'default': 'Detroit'}...",{},NaN,NaN,NaN,NaN,/gamecenter/edm-vs-det/2024/01/11/2023020640,https://www.ticketmaster.com/event/08005ED5C3A...
2,2023020659,20232024,2,2024-01-13,{'default': 'Little Caesars Arena'},False,2024-01-14T00:00:00Z,-05:00,-05:00,America/Detroit,FUT,OK,"[{'id': 351, 'market': 'H', 'countryCode': 'US...","{'id': 26, 'placeName': {'default': 'Los Angel...","{'id': 17, 'placeName': {'default': 'Detroit'}...",{},NaN,NaN,NaN,NaN,/gamecenter/lak-vs-det/2024/01/13/2023020659,https://www.ticketmaster.com/event/08005ED5C3A...
